In [ ]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from plyfile import PlyData
import zmsh

Download the Stanford bunny image.

In [ ]:
!wget --no-clobber http://graphics.stanford.edu/pub/3Dscanrep/bunny.tar.gz
!tar -zxvf bunny.tar.gz --wildcards --no-anchored '*bun_zipper_res2.ply'

Load in the model using the `plyfile` package and extract the points and triangles.

In [ ]:
filename = "bunny/reconstruction/bun_zipper_res2.ply"
with open(filename, "r") as bunny_file:
    model = PlyData.read(bunny_file)

vertices = model.elements[0]
x, y, z = vertices["x"], vertices["y"], vertices["z"]
points = np.column_stack((x, y, z))
triangles = np.array([tuple(elt) for elt in model.elements[1].data["vertex_indices"]])

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.view_init(elev=120)
ax.plot_trisurf(x, y, z, triangles=triangles);

Make a mapping from vertices to triangles.

In [ ]:
cotriangles = [[] for index in range(len(points))]
for index, triangle in enumerate(triangles):
    for vertex in triangle:
        cotriangles[vertex].append(index)

We'll pick out the first two vertices of triangle 0 to focus on.

In [ ]:
triangles[0]

In [ ]:
vertex0, vertex1 = triangles[0][:2]
vertex0, vertex1

In [ ]:
triangle_ids0 = cotriangles[vertex0]
triangle_ids1 = cotriangles[vertex1]
triangle_ids = list(set(triangle_ids0).union(triangle_ids1))
patch = triangles[triangle_ids]
patch

In [ ]:
vertex_ids = np.unique(patch.flatten())
vertex_ids

To simplify things, we'll renumber the vertices on this patch.

In [ ]:
id_map = np.vectorize({idx: val for val, idx in enumerate(vertex_ids)}.get)

In [ ]:
patch = id_map(patch)
vtx0 = id_map(vertex0)
vtx1 = id_map(vertex1)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
colors = ["tab:green" for index in range(len(vertex_ids))]
colors[vtx0] = "tab:orange"
colors[vtx1] = "tab:orange"
ax.plot_trisurf(x[vertex_ids], y[vertex_ids], z[vertex_ids], triangles=patch)
ax.scatter(x[vertex_ids], y[vertex_ids], z[vertex_ids], color=colors);

Now we'll convert this simplicial complex into its linear algebraic representation.

In [ ]:
d_0, d_1, d_2 = zmsh.polytopal.from_simplicial(patch)

We'll first collapse the edge between vertex 0 and vertex 1.
To do that, we can perform a row operation on the $d_1$ matrix.
The operation is to add the rows corresponding to vertex 0 and vertex 1 together.
We'll then zero out the row corresponding to vertex 1.

In [ ]:
P = np.eye(len(vertex_ids), dtype=np.int8)
P[vtx0, [vtx0, vtx1]] = (+1, +1)
P[vtx1, :] = 0
P

The code below shows the resulting matrix, which we'll call $e_1$.
You might notice that, for example, columns 0 and 1 are identical.
(So are columns 4 and 6 and columns 3 and 13.)
From a geometric viewpoint, this means that we have two edges that start at the same vertex and end at the same vertex.
This is undesirable.
The new 2-cells are the same as the old for now.

In [ ]:
e_0 = d_0
e_1 = P @ d_1
e_2 = d_2
print(e_1)

We might also have columns that are not exact duplicates by scalar multiples of each other.
For example, an edge that goes from vertex 0 to vertex 1, and another edge that goes from vertex 1 to vertex 0.
We can clean things up by:
1. replacing all 1-cells that are multiples of each other with a single representative
2. for any 2-cell that was incident upon any of the degenerate 1-cells, replace it with a 2-cell that is incident upon the new representative 1-cell.

The function below creates the matrices that we will multiply by $e_1$ and $e_2$ in order to merge these cells.

In [ ]:
A, B = zmsh.polytopal.make_reduction_matrices(e_1)

The matrix $A$ selects some of the columns of $e_1$.

In [ ]:
print(A)

Meanwhile the matrix $B$ does a row operation to collapse the incidence of any 2-cells to redundant edges down to the remaining non-redundant edges.

In [ ]:
print(B)

If we look at $e_1\cdot A$, we can see that there are no columns left that are multiples of each other.

In [ ]:
f_1 = e_1 @ A
print(f_1)

And the columns of $B\cdot e_2$ have their adjacencies to redundant columns summed together.

In [ ]:
print(B @ e_2)

In [ ]:
empty_cell_ids = np.flatnonzero(np.count_nonzero(B @ e_2, axis=0) < 3)
f_2 = np.delete(B @ e_2, empty_cell_ids, axis=1)
print(f_2)

You'll also note that some 2-cells are now empty, which we expect.

In [ ]:
ltriangles = zmsh.polytopal.to_simplicial([e_0, f_1, f_2])
print(ltriangles)

In [ ]:
vertex_ids[ltriangles]